In [1]:
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import cPickle, h5py, os, glob

from oc_ica import analysis, styles, plotting
reload(analysis)

<module 'oc_ica.analysis' from '/home/jesse/Development/oc_ica/oc_ica/analysis.pyc'>

# HDF5 Storage

In [259]:
base_folder = '/home/jesse/Development/results/oc_ica/'
n_iter = 10

In [260]:
n_mixtures = 32
OC = '2.0'
k = 2
priors = ['COHERENCE', 'INIT']
n_sources = int(n_mixtures * float(OC))
print n_mixtures, n_sources, OC, k
fit_folder = 'comparison_mixtures-{}_sources-{}_k-{}_priors-{}'.format(n_mixtures, n_sources, k, '_'.join(priors))
a_file = 'a_array-{}_OC-{}_priors-{}.h5'.format(n_mixtures, OC, '_'.join(priors))
print a_file
print fit_folder
fit_files = sorted(glob.glob(os.path.join(base_folder, fit_folder, 'comparison*.h5')))
sc_fits = None
models = [f.split('.')[-2].split('-')[-1] for f in fit_files]
print len(models), models

32 64 2.0 2
a_array-32_OC-2.0_priors-COHERENCE_INIT.h5
comparison_mixtures-32_sources-64_k-2_priors-COHERENCE_INIT
9 ['2', '4', 'COHERENCE_SOFT', 'COULOMB', 'COULOMB_F', 'RANDOM', 'RANDOM_F', 'SC', 'SM']


In [261]:
with h5py.File(os.path.join(base_folder, a_file), 'r') as f:
    A_array = f['A_array'].value
    A_priors = f['A_priors'].value

## Continue

In [262]:
all_colors = ['black', 'blue', 'red', 'green', 'purple', 'cyan']
seq_ftms = ['-', ':', '-.']
ver_ftms = ['-', '--']
model_color = {}
model_fmt = {}
color_idx = 1
int_idx = 0
for model in models:
    base_model = model.split('_')[0]
    if base_model.isdigit():
        model_color[model] = all_colors[0]
        model_fmt[model] = seq_ftms[int_idx]
        int_idx += 1
    elif '_' in model:
        if base_model in model_color.keys():
            model_color[model] = model_color[base_model]
            model_fmt[model] = ver_ftms[1]
        else:
            model_color[model] = all_colors[color_idx]
            model_fmt[model] = ver_ftms[0]
            color_idx += 1
    else:
        if model in [m.split('_')[0] for m in model_color.keys()]:
            eqiv = [m for m in model_color.keys() if model == m.split('_')[0]][0]
        else:
            model_color[model] = all_colors[color_idx]
            model_fmt[model] = ver_ftms[0]
            color_idx += 1

In [263]:
with h5py.File(os.path.join(base_folder, fit_folder, fit_files[0]), 'r') as f:
    lambdas = f['lambdas'].value
n_mixtures, n_sources = A_array.shape[2:]
n_iter = A_array.shape[1]

In [264]:
W_fits = np.full((len(A_priors), len(models), lambdas.size, n_iter, n_sources, n_mixtures),
                 np.nan, dtype='float32')
results = np.full((len(A_priors), len(models), lambdas.size, n_iter, 2),
                  np.nan, dtype='float32')
null_results = np.full((len(A_priors), len(models), lambdas.size, (n_iter**2-n_iter)//2, 2),
                  np.nan, dtype='float32')

In [265]:
for ii, f_name in enumerate(fit_files):
    with h5py.File(os.path.join(base_folder, fit_folder, f_name), 'r') as f:
        W_fits[:, ii] = np.squeeze(f['W_fits'])[:, :, :n_iter]

if sc_fits is not None:
    loc = 0
    for ii, f_name in enumerate(sc_fits):
        with h5py.File(os.path.join(base_folder, fit_folder, f_name), 'r') as f:
            n_lambdas = f['W_fits'].shape[2]
            W_fits[:, -1, loc:loc+n_lambdas, :10] = np.squeeze(f['W_fits'])[:, :, :n_iter]
            loc += n_lambdas

In [266]:
reload(analysis)
results_file = os.path.join(base_folder, fit_folder, 'results.h5')
if False: #os.path.exists(results_file):
    with h5py.File(results_file) as f:
        results = f['results'].value
else:
    for ii, p in enumerate(A_priors):
        for jj, m in enumerate(models):
            for kk, l in enumerate(lambdas):
                for ll in range(n_iter):
                    try:
                        A = A_array[ii, ll]
                        W = W_fits[ii, jj, kk, ll]
                        assert (not np.isnan(A.sum())) and (not np.isnan(W.sum()))
                        results[ii, jj, kk, ll] = analysis.recovery_statistics_AW(A, W)
                        #results[ii, jj, kk, ll] = [hd, pd, mod]
                        #max_angles[ii, jj, kk, ll] = ma
                        #other_angles[ii, jj, kk, ll] = oa
                    except (ValueError, AssertionError):
                        pass
    with h5py.File(results_file, 'w') as f:
        f.create_dataset('results', data=results)

In [267]:
results_file = os.path.join(base_folder, fit_folder, 'null_results.h5')
if False: #os.path.exists(results_file):
    with h5py.File(results_file) as f:
        null_results = f['null_results'].value
else:
    for ii, p in enumerate(A_priors):
        for jj, m in enumerate(models):
            for kk, l in enumerate(lambdas):
                loc = 0
                for ll in range(n_iter):
                    for mm in range(ll+1, n_iter):
                        try:
                            A = A_array[ii, ll]
                            W = W_fits[ii, jj, kk, mm]
                            assert (not np.isnan(A.sum())) and (not np.isnan(W.sum()))
                            null_results[ii, jj, kk, loc] = analysis.recovery_statistics_AW(A, W)
                            loc += 1
                        except (ValueError, AssertionError):
                            pass
    with h5py.File(results_file, 'w') as f:
        f.create_dataset('null_results', data=null_results)

# Summary Plots

In [268]:
n_mixtures = 32
OCs = ['1.0', '1.5', '2.0', '2.05', '2.25', '2.5', '2.75', '3.0', '4.0', '8.0']
keep_OCs = ['1.0', '1.5', '2.0', '2.25', '2.5', '2.75', '3.0', '4.0', '8.0']
k = 2

keep_models = ['2', '4', 'COHERENCE_SOFT', 'COULOMB', 'COULOMB_F', 'RANDOM', 'RANDOM_F', 'SC', 'SM']

ks = [2, 4, 8, 16]

labelpad = -3

n_prior = 1
f = plt.figure(figsize=(10, 8))
ae = plt.subplot2grid((3, 3), (0, 0))
am = plt.subplot2grid((3, 3), (0, 1))
ap = plt.subplot2grid((3, 3), (0, 2))
p = A_priors[n_prior]
print('prior: {}'.format(p))
ii = n_prior
for jj, m in enumerate(keep_models):
    jjp = models.index(m)
    if m in styles.models:
        fmt = styles.line_styles[m]

        color = styles.colors[m]
        label = styles.labels[m]

        if m == 'SM':
            delta = np.tile(np.nanmean(results[ii, jjp, 0, :, 0]), lambdas.size)
            mma = np.tile(np.nanmean(results[ii, jjp, 0, :, 1]), lambdas.size)
            null_delta = np.tile(np.nanmean(null_results[ii, jjp, 0, :, 0]), lambdas.size)
            null_mma = np.tile(np.nanmean(null_results[ii, jjp, 0, :, 1]), lambdas.size)
        else:
            delta = np.nanmean(results[ii, jjp, :, :, 0], axis=1)
            mma = np.nanmean(results[ii, jjp, :, :, 1], axis=1)
            null_delta = np.nanmean(null_results[ii, jjp, :, :, 0], axis=1)
            null_mma = np.nanmean(null_results[ii, jjp, :, :, 1], axis=1)

        ae.semilogx(lambdas, delta, fmt, label=label, c=color)
        ae.semilogx(lambdas, delta, '.', c=color)
        ae.minorticks_off()
        am.semilogx(lambdas, mma, fmt, label=label, c=color)
        am.semilogx(lambdas, mma, '.', c=color)
        am.minorticks_off()
        ap.semilogx(lambdas, delta/null_delta + mma/null_mma, fmt, label=label, c=color)
        ap.semilogx(lambdas, delta/null_delta + mma/null_mma, '.', c=color)
        ap.minorticks_off()
    
#ae.legend(loc='best', ncol=2, prop={'size': 8}, frameon=False)
#ae.set_title(p)
ae.set_ylabel(r'$\Delta P$', labelpad=labelpad, fontsize=12)
ae.set_ylim([0, np.nanmax(results[...,0])])
#am.set_title(p)
am.set_ylabel(r'median($p_{\mathrm{min}}$)', labelpad=labelpad, fontsize=12)
am.set_ylim([0, 80])
#ap.set_title(p)
ap.set_ylabel('Normalized Sum', labelpad=labelpad, fontsize=12)
ap.set_ylim([0, 2.5])
    
ae.set_xlabel(r'$\lambda$', labelpad=labelpad, fontsize=12)
am.set_xlabel(r'$\lambda$', labelpad=labelpad, fontsize=12)
ap.set_xlabel(r'$\lambda$', labelpad=labelpad, fontsize=12)

ax1 = plt.subplot2grid((3, 3), (1, 0), colspan=3)
ax2 = plt.subplot2grid((3, 3), (2, 0), colspan=3)

x = np.array([float(oc) for oc in keep_OCs])
y = np.zeros((len(keep_models), x.size))
y_std = np.zeros_like(y)
for ii, OC in enumerate(keep_OCs):
    iik = OCs.index(OC)
    n_sources = int(n_mixtures * float(OC))
    fit_folder = 'comparison_mixtures-{}_sources-{}_k-{}_priors-{}'.format(n_mixtures, n_sources, k, '_'.join(priors))
    results_file = os.path.join(base_folder, fit_folder, 'results.h5')
    null_results_file = os.path.join(base_folder, fit_folder, 'null_results.h5')
    with h5py.File(results_file) as f:
        results = f['results'].value
    with h5py.File(null_results_file) as f:
        null_results = f['null_results'].value
    for kk, m in enumerate(keep_models):
        kkp = models.index(m)
        if m == 'SM':
            mean_null = np.nanmean(null_results[n_prior, kkp, 0], axis=0, keepdims=True)
            r = results[n_prior, kkp, 0] / mean_null
            pos = np.nanmean(r, axis=0).sum()
            std = np.nanstd(r.sum(axis=-1))
        else:
            mean_null = np.nanmean(null_results[n_prior, kkp], axis=1, keepdims=True)
            r = results[n_prior, kkp] / mean_null
            r_mean = np.nanmean(r, axis=1).sum(axis=1)
            r_min_idx = r_mean.argmin()
            pos = np.nanmean(r[r_min_idx], axis=0).sum()
            std = np.nanstd(r[r_min_idx].sum(axis=-1))
        y[kk, ii] = pos
        y_std[kk, ii] = std

for ym, y_stdm, m in zip(y, y_std, keep_models):
    fmt = styles.line_styles[m]
    color = styles.colors[m]
    label = styles.labels[m]
    ax1.errorbar(x, ym, yerr=y_stdm, fmt=fmt, color=color, label=label)
ax1.set_ylim(0, 2)
ax1.set_xlim(.75, 4.25)
ax1.set_xticks(np.linspace(1, 4, 4))
ax1.set_yticks(np.linspace(0, 2, 3))
ax1.set_xlabel('Overcompleteness', labelpad=labelpad, fontsize=12)
ax1.set_ylabel('Normalized Sum', labelpad=labelpad, fontsize=12)


OC=2.
x = np.array(ks)
y = np.zeros((len(keep_models), x.size))
y_std = np.zeros_like(y)
for ii, k in enumerate(ks):
    n_sources = int(n_mixtures * float(OC))
    fit_folder = 'comparison_mixtures-{}_sources-{}_k-{}_priors-{}'.format(n_mixtures, n_sources, k, '_'.join(priors))
    results_file = os.path.join(base_folder, fit_folder, 'results.h5')
    null_results_file = os.path.join(base_folder, fit_folder, 'null_results.h5')
    print results_file
    with h5py.File(results_file) as f:
        results = f['results'].value
    print null_results_file
    with h5py.File(null_results_file) as f:
        null_results = f['null_results'].value
    for kk, m in enumerate(keep_models):
        kkp = models.index(m)
        if m == 'SM':
            mean_null = np.nanmean(null_results[n_prior, kkp, 0], axis=0, keepdims=True)
            r = results[n_prior, kkp, 0] / mean_null
            pos = np.nanmean(r, axis=0).sum()
            std = np.nanstd(r.sum(axis=-1))
        else:
            mean_null = np.nanmean(null_results[n_prior, kkp], axis=1, keepdims=True)
            r = results[n_prior, kkp] / mean_null
            r_mean = np.nanmean(r, axis=1).sum(axis=1)
            r_min_idx = r_mean.argmin()
            pos = np.nanmean(r[r_min_idx], axis=0).sum()
            std = np.nanstd(r[r_min_idx].sum(axis=-1))
        y[kk, ii] = pos
        y_std[kk, ii] = std

for ym, y_stdm, m in zip(y, y_std, keep_models):
    fmt = styles.line_styles[m]
    color = styles.colors[m]
    label = styles.labels[m]
    ax2.errorbar(x, ym, yerr=y_stdm, fmt=fmt, color=color, label=label)
ax2.legend(loc='best', ncol=2, frameon=False)
ax2.set_xlim(1, 17)
ax2.set_xticks(np.linspace(2, 16, 8))
ax2.set_yticks(np.linspace(0, 2, 3))
ax2.set_xlabel(r'$k$-sparseness', labelpad=labelpad, fontsize=12)
ax2.set_ylabel('Normalized Sum', labelpad=labelpad, fontsize=12)
    
plt.tight_layout()
plt.show()

prior: INIT
/home/jesse/Development/results/oc_ica/comparison_mixtures-32_sources-64_k-2_priors-COHERENCE_INIT/results.h5
/home/jesse/Development/results/oc_ica/comparison_mixtures-32_sources-64_k-2_priors-COHERENCE_INIT/null_results.h5
/home/jesse/Development/results/oc_ica/comparison_mixtures-32_sources-64_k-4_priors-COHERENCE_INIT/results.h5
/home/jesse/Development/results/oc_ica/comparison_mixtures-32_sources-64_k-4_priors-COHERENCE_INIT/null_results.h5
/home/jesse/Development/results/oc_ica/comparison_mixtures-32_sources-64_k-8_priors-COHERENCE_INIT/results.h5
/home/jesse/Development/results/oc_ica/comparison_mixtures-32_sources-64_k-8_priors-COHERENCE_INIT/null_results.h5
/home/jesse/Development/results/oc_ica/comparison_mixtures-32_sources-64_k-16_priors-COHERENCE_INIT/results.h5
/home/jesse/Development/results/oc_ica/comparison_mixtures-32_sources-64_k-16_priors-COHERENCE_INIT/null_results.h5


# A Check

In [3]:
a_file = 'a_array-32_OC-2.0_priors-COHERENCE_INIT.h5'
with h5py.File(os.path.join(base_folder, a_file), 'r') as f:
    A_array = f['A_array'].value
    A_priors = f['A_priors'].value

In [7]:
reload(plotting)
plotting.plot_angles_1column(np.transpose(A_array, (0, 1, 3, 2)), 1, A_priors, plot_init=False)
plt.show()

In [30]:
A_array.shape

(2, 10, 32, 64)

In [91]:
results.shape

(7, 11, 17, 10, 2)

In [92]:
results[0, -1]

array([[[ 54.        ,  37.72133636],
        [ 48.        ,  38.90277863],
        [ 52.        ,  37.9727478 ],
        [ 52.        ,  37.98392487],
        [ 56.        ,  38.73665619],
        [ 58.        ,  39.25117493],
        [ 56.        ,  38.4540863 ],
        [ 48.        ,  39.203125  ],
        [ 64.        ,  39.57582092],
        [ 52.        ,  38.31627655]],

       [[         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan]],

       [[         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        [         nan,          nan],
        